# Dependencies

In [29]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('./')
import numpy as np
from holisticai.robustness.utils import AttackDataset
from holisticai.robustness.utils import load_diabetes
from holisticai.robustness.utils import train_holisticai_regressor
from holisticai.robustness.utils import train_sklearn_regressor
from holisticai.wrappers.regression import ScikitlearnRegressor
from holisticai.wrappers.classification.scikitlearn import ScikitlearnDecisionTreeRegressor
from holisticai.robustness.metrics import classification_metrics
from holisticai.robustness.mitigation.attacks.inference import (
    AttributeInferenceBaseline,AttributeInferenceBaselineTrueLabel,AttributeInferenceMembership,AttributeInferenceWhiteBoxLifestyleDecisionTree,AttributeInferenceWhiteBoxDecisionTree,AttributeInferenceBlackBox
)

np.random.seed(100)

# Load Dataset

In [30]:
(x_train, y_train), (x_test, y_test) = load_diabetes(test_set=0.5)

## Baseline Attribute Inference on Diabetes Dataset:
Implements an attribute inference attack on the Diabetes dataset using the baseline method that doesn't consider the true label during training. The specific feature to infer is denoted as 'social'.

In [31]:
dataset = AttackDataset(x=x_train, attack_train_ratio=0.5)
attack_feature = 1  # social

attack = AttributeInferenceBaseline(attack_feature=attack_feature)
x = dataset.attribute_inference_train()
attack.fit(x)

x = dataset.attribute_inference_test()
attack_x = np.delete(x, attack_feature, 1)
feat_true = x[:, attack_feature]

values = [-0.88085106,  1.]
feat_pred = attack.infer(attack_x, values=values)
df = classification_metrics(y_true=feat_true,
                            y_pred=feat_pred,
                            positive_value=values[1])
print(df)

             Values  Reference
Accuracy   0.558559          1
Precision  0.566038          1
Recall     0.535714          1


## Baseline Attribute Inference with True Labels on Diabetes Dataset:
Enhances the baseline attribute inference attack by incorporating true labels into the training, offering potentially improved inference accuracy for the 'social' feature.

In [32]:
dataset = AttackDataset(x=x_train, y=y_train, attack_train_ratio=0.5)
attack_feature = 1  # social

attack = AttributeInferenceBaselineTrueLabel(attack_feature=attack_feature, is_regression=True)
x , y = dataset.attribute_inference_train()
attack.fit(x, y)

x , y = dataset.attribute_inference_test()
attack_x = np.delete(x, attack_feature, 1)
feat_true = x[:, attack_feature]


#values = [-0.93853666,  1.]
values = [-0.88085106,  1.]
feat_pred = attack.infer(attack_x, y, values=values)
df = classification_metrics(y_true=feat_true,
                            y_pred=feat_pred,
                            positive_value=values[1])
print(df)

             Values  Reference
Accuracy   0.540541          1
Precision  0.523364          1
Recall     1.000000          1


## Black Box Attribute Inference on Diabetes Dataset:
Executes a black box attribute inference attack, which uses predictions from a trained regression model to infer the 'social' feature values within the Diabetes dataset.

In [33]:
dataset = AttackDataset(x=x_train, y=y_train, attack_train_ratio=0.5)
attack_feature = 1  # social

regressor = train_holisticai_regressor(x_train, y_train)
regressor = ScikitlearnRegressor(regressor)

attack = AttributeInferenceBlackBox(estimator=regressor, attack_feature=attack_feature, scale_range=(0,1))
x , y = dataset.attribute_inference_train()
pred = regressor.predict(x)
attack.fit(x, y, pred)

x , y = dataset.attribute_inference_test()
pred = regressor.predict(x)
attack_x = np.delete(x, attack_feature, 1)
feat_true = x[:, attack_feature]

values = [-0.88085106,  1.]
feat_pred = attack.infer(attack_x, y, pred, values=values)
df = classification_metrics(y_true=feat_true,
                            y_pred=feat_pred,
                            positive_value=values[1])
print(df)

             Values  Reference
Accuracy   0.567568          1
Precision  0.566667          1
Recall     0.607143          1


## White Box Lifestyle Attribute Inference with Decision Tree on Diabetes Dataset:
Performs a white box attribute inference attack using a decision tree regressor, implying the attacker has access to the model's structure and parameters to infer the 'social' feature.

In [34]:
dataset = AttackDataset(x=x_train, y=y_train, attack_train_ratio=0.5)
attack_feature = 1  # social

regressor = train_sklearn_regressor(x_train, y_train)
regressor = ScikitlearnDecisionTreeRegressor(regressor)
attack = AttributeInferenceWhiteBoxLifestyleDecisionTree(estimator=regressor, attack_feature=attack_feature)

x , y = dataset.attribute_inference_test()
attack_x = np.delete(x, attack_feature, 1)
feat_true = x[:, attack_feature]

values = [-0.88085106,  1.]
priors = [3465 / 5183, 1718 / 5183]
feat_pred = attack.infer(attack_x,  values=values, priors=priors)
df = classification_metrics(y_true=feat_true,
                            y_pred=feat_pred,
                            positive_value=values[1])
print(df)

             Values  Reference
Accuracy   0.522523          1
Precision  0.800000          1
Recall     0.071429          1
